# Imports

##### General imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import pprint

##### Import from flatland environment 

In [3]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

##### Import from our framework

In [4]:
from src.graph import NetworkGraph

In [5]:
from src.flows import *
from src.navigation.navigation_path import *

# Test of time expanded network

##### Create a flatland network

sparse netwrok

In [6]:
if True:
    number_agents = 20

    size_side = 50
    stochastic_data = {'prop_malfunction': 0,  # Percentage of defective agents
                       'malfunction_rate': 30,  # Rate of malfunction occurence
                       'min_duration': 3,  # Minimal duration of malfunction
                       'max_duration': 20  # Max duration of malfunction
                       }
    speed_ration_map = {1.: 1,  # Fast passenger train
                        1. / 2.: 0,  # Fast freight train
                        1. / 3.: 0,  # Slow commuter train
                        1. / 4.: 0}  # Slow freight train
    env = RailEnv(width=size_side,
                  height=size_side,
                  rail_generator=sparse_rail_generator(max_num_cities=number_agents+2
                                                       
                                                       ,
                                                       # Number of cities in map (where train stations are)
                                                       seed=21,  # Random seed
                                                       grid_mode=True,
                                                       max_rails_between_cities=4,
                                                       max_rails_in_city=4,
                                                       ),
                  schedule_generator=sparse_schedule_generator(speed_ration_map,seed = 2),
                  number_of_agents=number_agents,
                  stochastic_data=stochastic_data,  # Malfunction data generator
                  obs_builder_object=GlobalObsForRailEnv(),
                  remove_agents_at_target=True
                  )

    # RailEnv.DEPOT_POSITION = lambda agent, agent_handle : (agent_handle % env.height,0)
    env.reset()

    env_renderer = RenderTool(env,
                              agent_render_variant=AgentRenderVariant.AGENT_SHOWS_OPTIONS_AND_BOX,
                              show_debug=True,
                              screen_height=1100,
                              screen_width=1800)
    env_renderer.reset()

    env_renderer.render_env(show=True, show_observations=False, show_predictions=False)


modify the starting point so that all agents have different starting point

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import pprint

from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

from src.flows import *

number_agents = 2

size_side = 20
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=number_agents, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()



env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)

solver = Solver("test.log",verbose=False)

solver.solve(env,100)

env.agents

solver.master.get_solution()

solver.solution

solver.run(env,env_renderer)

In [ ]:
flatlandNetwork = NetworkGraph(np.array(env.rail.grid.tolist()))

In [ ]:
env.restart_agents()

In [ ]:
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)


In [ ]:
env.agents[0].status

In [ ]:
env.agents

solver.initialSolutionGenerator.stats

In [ ]:
solver.solution

In [ ]:
from src.navigation.navigation_path import *

In [ ]:
walk_path(env,env_renderer,solver.solution[0],0)

In [ ]:
solver.initialSolutionGenerator.stats

In [ ]:
solver2 = Solver("test.log","Arc Formulation")
solver2.solve(env,100)

In [ ]:
solver2.stats

In [ ]:
solver.stats

In [ ]:
master = solver.master 

In [ ]:
result = np.where(np.array(solutions) == 1.)

In [ ]:
solutions = master.model.getAttr("X")

In [ ]:
result

In [ ]:
for v in master.model.getVars():
        print('%s %g' % (v.varName, v.x))

In [ ]:
inter = v.VarName.split("[")[-1]
inter = inter.split("]")[0]
c = int(inter.split(",")[0])
path = int(inter.split(",")[-1])

In [ ]:
c,path

In [ ]:
sources = []
sinks = []
directions = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)
    try:
        directions.append(agent.direction)
    except:
        pass

In [ ]:
sources

In [ ]:
sinks

In [ ]:
directions

In [ ]:
solver.master.get_solution()

In [ ]:
if len(directions) == 0:
    directions = None

##### create a time expanded network

In [ ]:
import time

In [ ]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=200)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

In [ ]:
TestNetworkTime.connect_sources_and_sink(sources,sinks,directions)

### create initial solution

In [ ]:
from tqdm import tqdm

In [ ]:
constraints, find_constraints = TestNetworkTime.get_topology_network()

In [ ]:
test = InitialSolutionGenerator(TestNetworkTime,constraints,find_constraints,len(sources))

In [ ]:
solution = test.getInitialSolution()

In [ ]:
test.showStats()

In [ ]:
solution

In [ ]:
len(solution)

In [ ]:
cost = 0
for p in solution:
    cost += len(p)
    
print(cost)

In [11]:
solver = Solver("./test_sparse.log",useDirections=True)

In [12]:
solver.solve(env)

Had to drop multiple agents, conflict with their starting position
[6, 9, 13, 14, 15, 18, 19]


13it [06:09, 28.44s/it]

Academic license - for non-commercial use only


score: 1983.0


1983.0

In [13]:
solver.run(env,env_renderer)

Error ! agent  6 is supposed to be at cell (33, 17) but is at (6, 46) at time 1
Error ! agent  7 is supposed to be at cell (31, 5) but is at (33, 17) at time 1
Error ! agent  8 is supposed to be at cell (19, 20) but is at (31, 5) at time 1
Error ! agent  9 is supposed to be at cell (43, 46) but is at (6, 30) at time 1
Error ! agent  10 is supposed to be at cell (45, 44) but is at (19, 20) at time 1
Error ! agent  11 is supposed to be at cell (6, 7) but is at (43, 46) at time 1
Error ! agent  6 is supposed to be at cell (33, 17) but is at (6, 46) at time 2
Error ! agent  7 is supposed to be at cell (31, 4) but is at (33, 16) at time 2
Error ! agent  8 is supposed to be at cell (19, 19) but is at (31, 4) at time 2
Error ! agent  9 is supposed to be at cell (42, 46) but is at (5, 30) at time 2
Error ! agent  10 is supposed to be at cell (46, 44) but is at (19, 19) at time 2
Error ! agent  11 is supposed to be at cell (6, 8) but is at (42, 46) at time 2
Error ! agent  6 is supposed to be a

Error ! agent  1 is supposed to be at cell (1, 31) but is at (0, 36) at time 20
Error ! agent  6 is supposed to be at cell (31, 12) but is at (1, 44) at time 20
Error ! agent  7 is supposed to be at cell (33, 0) but is at (30, 11) at time 20
Error ! agent  8 is supposed to be at cell (19, 29) but is at (42, 1) at time 20
Error ! agent  9 is supposed to be at cell (26, 44) but is at (0, 35) at time 20
Error ! agent  10 is supposed to be at cell (45, 33) but is at (31, 21) at time 20
Error ! agent  11 is supposed to be at cell (16, 6) but is at (27, 45) at time 20
Error ! agent  1 is supposed to be at cell (1, 32) but is at (0, 36) at time 21
Error ! agent  6 is supposed to be at cell (31, 11) but is at (1, 43) at time 21
Error ! agent  7 is supposed to be at cell (33, 0) but is at (30, 11) at time 21
Error ! agent  8 is supposed to be at cell (19, 30) but is at (43, 1) at time 21
Error ! agent  9 is supposed to be at cell (25, 44) but is at (0, 35) at time 21
Error ! agent  10 is suppos

Error ! agent  1 is supposed to be at cell (6, 44) but is at (0, 36) at time 38
Error ! agent  6 is supposed to be at cell (36, 23) but is at (1, 26) at time 38
Error ! agent  7 is supposed to be at cell (44, 4) but is at (20, 6) at time 38
Error ! agent  9 is supposed to be at cell (11, 45) but is at (0, 35) at time 38
Error ! agent  10 is supposed to be at cell (39, 21) but is at (31, 4) at time 38
Error ! agent  11 is supposed to be at cell (29, 11) but is at (19, 35) at time 38
Error ! agent  1 is supposed to be at cell (7, 44) but is at (0, 36) at time 39
Error ! agent  6 is supposed to be at cell (36, 22) but is at (1, 25) at time 39
Error ! agent  7 is supposed to be at cell (44, 5) but is at (19, 6) at time 39
Error ! agent  9 is supposed to be at cell (10, 45) but is at (0, 35) at time 39
Error ! agent  10 is supposed to be at cell (39, 20) but is at (31, 3) at time 39
Error ! agent  11 is supposed to be at cell (30, 11) but is at (19, 34) at time 39
Error ! agent  1 is suppos

Error ! agent  1 is supposed to be at cell (24, 44) but is at (0, 36) at time 56
Error ! agent  5 is supposed to be at cell (23, 23) but is at (23, 22) at time 56
Error ! agent  6 is supposed to be at cell (49, 18) but is at (13, 20) at time 56
Error ! agent  7 is supposed to be at cell (49, 17) but is at (9, 1) at time 56
Error ! agent  9 is supposed to be at cell (0, 38) but is at (0, 35) at time 56
Error ! agent  10 is supposed to be at cell (49, 23) but is at (46, 1) at time 56
Error ! agent  11 is supposed to be at cell (32, 26) but is at (23, 23) at time 56
Error ! agent  1 is supposed to be at cell (25, 44) but is at (0, 36) at time 57
Error ! agent  5 is supposed to be at cell (22, 23) but is at (23, 22) at time 57
Error ! agent  6 is supposed to be at cell (49, 19) but is at (13, 19) at time 57
Error ! agent  7 is supposed to be at cell (49, 18) but is at (8, 1) at time 57
Error ! agent  9 is supposed to be at cell (0, 37) but is at (0, 35) at time 57
Error ! agent  10 is supp

Error ! agent  4 is supposed to be at cell (19, 24) but is at (23, 21) at time 74
Error ! agent  5 is supposed to be at cell (19, 37) but is at (23, 22) at time 74
Error ! agent  6 is supposed to be at cell (45, 32) but is at (26, 21) at time 74
Error ! agent  7 is supposed to be at cell (36, 22) but is at (3, 13) at time 74
Error ! agent  10 is supposed to be at cell (45, 37) but is at (49, 16) at time 74
Error ! agent  4 is supposed to be at cell (19, 25) but is at (23, 21) at time 75
Error ! agent  5 is supposed to be at cell (19, 38) but is at (23, 22) at time 75
Error ! agent  6 is supposed to be at cell (45, 33) but is at (26, 22) at time 75
Error ! agent  7 is supposed to be at cell (36, 23) but is at (2, 13) at time 75
Error ! agent  10 is supposed to be at cell (45, 38) but is at (49, 17) at time 75
Error ! agent  4 is supposed to be at cell (19, 26) but is at (23, 21) at time 76
Error ! agent  5 is supposed to be at cell (19, 39) but is at (23, 22) at time 76
Error ! agent  6

Error ! agent  4 is supposed to be at cell (25, 32) but is at (23, 21) at time 96
Error ! agent  6 is supposed to be at cell (44, 45) but is at (31, 8) at time 96
Error ! agent  7 is supposed to be at cell (31, 15) but is at (13, 21) at time 96
Error ! agent  10 is supposed to be at cell (38, 44) but is at (40, 18) at time 96
Error ! agent  4 is supposed to be at cell (26, 32) but is at (23, 21) at time 97
Error ! agent  7 is supposed to be at cell (31, 16) but is at (13, 20) at time 97
Error ! agent  10 is supposed to be at cell (37, 44) but is at (40, 18) at time 97
Error ! agent  4 is supposed to be at cell (27, 32) but is at (23, 21) at time 98
Error ! agent  7 is supposed to be at cell (31, 17) but is at (13, 19) at time 98
Error ! agent  10 is supposed to be at cell (36, 44) but is at (40, 18) at time 98
Error ! agent  4 is supposed to be at cell (28, 32) but is at (23, 21) at time 99
Error ! agent  7 is supposed to be at cell (31, 18) but is at (14, 19) at time 99
Error ! agent 

In [ ]:
env_renderer.render_env(show = True,show_predictions=False,show_observations= False)

In [ ]:
env.agents

In [ ]:
solver.solution_cell[0][0] = (9,8)

In [ ]:
_ = env.step({0:4,1:2,2:4})
_ = env.step({0:4,1:2,2:4})
_ = env.step({0:1,1:4,2:1})

In [ ]:
 _ = env.step({i:1 for i in range(3)})

In [ ]:
env_renderer.render_env(show=True, 
                            show_predictions=False, 
                            show_observations=False)

In [ ]:
for k in range(number_agents):
    _ = env.step({k:1})
    env_renderer.render_env(show=True, 
                            show_predictions=False, 
                            show_observations=False)
    #_ = env.step({k:actions_list[k][0]})
    env_renderer.render_env(show=True, 
                            show_predictions=False, 
                            show_observations=False)

In [ ]:
_ = env.step({})
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)

In [ ]:
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)

In [ ]:
actions_list = [actions_for_path(path, env.agents[k].direction) for k, path in solver.solution_cell.items()]

In [ ]:
setup_env(env,[p for _,p in solver.solution_cell.items()],actions_list)

In [ ]:
check_position(1,env,[p for _,p in solver.solution_cell.items()])

In [ ]:
i = 1

In [ ]:
actions_dict = {k:actions_list[k][i] for k in range(3) if len(actions_list[k])>i}
env.step(actions_dict)
check_position(i+1,env,[p for _,p in solver.solution_cell.items()])
env_renderer.render_env(show=True, show_predictions=False, 
                        show_observations=False)
i += 1

In [ ]:
env.agents

In [ ]:
i

In [ ]:
env.agents[0].position

In [ ]:
solver.solution_cell[0]

In [ ]:
[len(p) for p in actions_list]

In [ ]:
walk_path(env,env_renderer,path_faulty,1)

In [ ]:
path_faulty = solver.solution_cell[1]

In [ ]:
from src.navigation.navigation_path import *

In [ ]:
actions = actions_for_path(path_faulty,env.agents[1].direction)

In [ ]:
print(actions)

In [ ]:
_= env.step({1:1})
i = 0

In [ ]:
_ = env.step({1:actions[i]})
print(actions[i])
env_renderer.render_env(show=True, 
                        show_predictions=False, 
                        show_observations=False)
i += 1

In [ ]:
solver.solution_cell

In [ ]:
cost = 0
for p in solver.initialSolutionGenerator.solution:
    cost += len(p)
    
print(cost)

In [ ]:
env.agents

In [ ]:
solver.solution_cell

In [ ]:
env.agents

In [ ]:
solver.solution_cell[]

In [ ]:
longest = 0
for agent,path in solver.solution_cell.items():
        longest = max(longest,len(path))


In [ ]:
pathsSameSize = []

In [ ]:
for agent,path in solver.solution_cell.items():
    for i in range(longest-len(path)):
        path.append(None)
    pathsSameSize.append(path)

In [ ]:
for i in range(longest-1):
    cell_used = []
    cell_to_be_used = []
    for p in pathsSameSize:
        cell_used.append(p[i])
        cell_to_be_used.append(p[i+1])
    if len(set(cell_used)) != len(cell_used) and None not in cell_used:
        print("error at time ",i)
        print(cell_used)
    usedattwotimestep = set(cell_used).intersection(cell_to_be_used)
    if len(usedattwotimestep)>0:
        for i,elt in enumerate(cell_used):
            for j,elt2 in enumerate(cell_to_be_used):
                if i!= j and elt2 == elt and elt2 is not None:
                    print("swapping")
                    print(cell_used)
                    print(cell_to_be_used)
                    print("")

In [ ]:
from src.flows.PricingProblem import PricingSolver

In [ ]:
start = time.time()
flag = True
iteration = 1
master = MasterProblem.MasterProblem(solution,constraints,find_constraints,number_agents)
master.build()
pricingSolver = PricingSolver(TestNetworkTime.graph,constraints,find_constraints,len(sources))
while flag:
    print("iteration ",iteration)
    master.solveRelaxedModel()
    duals = master.getDualVariables()
    pathsToAdd, flag = pricingSolver.get_columns_to_add(master.getDualVariables(),master.constraintsActivated)
    if flag:
        #pprint.pprint(pathsToAdd)
        print(f" commodities with updated paths: {pathsToAdd.keys()}")
        iteration+= 1
        master.addColumn(pathsToAdd)
print(f"column generation method took {np.round(time.time()-start,3)} sec")

In [ ]:
master.relaxedModel.objVal

In [ ]:
master.model.optimize()

In [ ]:
master.model.objVal

In [ ]:
for v in master.relaxedModel.getVars():
    print('%s %g' % (v.varName, v.x))

In [ ]:
for v in master.model.getVars():
    print('%s %g' % (v.varName, v.x))

In [ ]:
master.cost

In [ ]:
master.model.write("temp/test.lp")

##### Test LP Formulation

test a simple graph

In [ ]:
import datetime as dt
print(dt.datetime.now())

In [ ]:
constraints,findConstraints = TestNetworkTime.get_topology_network()

In [ ]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),constraints,integer = True)

In [ ]:
mcflow.solve()

In [ ]:
paths = solver2.mcflow.solution_complete_edges

In [ ]:
score = 0
for _,path in paths.items():
    score += len(path)
    
print(f"score {score}")

In [ ]:
mcflow = solver2.mcflow

In [ ]:
paths

In [ ]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

In [ ]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths

In [ ]:
colors = [(230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48),
          (145, 30, 180), (70, 240, 240), (240, 50, 230), (210, 245, 60), (250, 190, 190),
          (0, 128, 128), (230, 190, 255), (170, 110, 40), (255, 250, 200), (128, 0, 0), (170, 255, 195),
          (128, 128, 0), (255, 215, 180), (0, 0, 128), (128, 128, 128), (255, 255, 255), (0, 0, 0)]
colors = [(x[0]/255.,x[1]/255.,x[2]/255.) for x in colors]

In [ ]:
solver.transitionNetwork.show(paths = mcflow.solution_complete_edges)